In [ ]:
# --- DeepSeek Prompt Chain for Cybersecurity EDA (DeepSeek via OpenAI SDK) ---

import os, io, time, traceback
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
from typing import Dict, List
from openai import OpenAI

# 1. Environment / API Setup --------------------------------------------------
# Expect an env file containing DEEPSEEK_API_KEY
#ENV_PATH = r"/Users/sarahsetiawan/Desktop/VNFCDR-1/SarahCode/Generative_AI/GEMINI_API_KEY.env"  # <-- update if needed to point to DeepSeek key file
ENV_PATH = r"C:\\Users\\macdo\\Github\\VNFCDR-1\\Chris Code\\Generative AI Data Cleaning\\Machine Learning\\Step 3_DeepSeek Prompting\\DEEPSEEK_API_KEY.env"
load_dotenv(ENV_PATH)
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise RuntimeError("DEEPSEEK_API_KEY not found. Ensure the env file exists and key is set.")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")
print("DeepSeek API client configured successfully.")

# 2. Data Load ----------------------------------------------------------------
#DATA_PATH = r"/Users/sarahsetiawan/Desktop/VNFCDR-1/SarahCode/Sample_df/Code/MachineLearning/SampleData_API/CSVs/Representative_APISample_20000_2.csv"
DATA_PATH = r"E:\\Datasets\\UNSW-NB15\\Training and Testing Sets\\UNSW_NB15_concatenated_dropped.csv"
try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
except FileNotFoundError:
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}")
print(f"Dataset loaded successfully. Shape: {df.shape}")

# 3. Context Preparation (Token-Optimized) ------------------------------------


def build_schema_summary(dataframe: pd.DataFrame) -> str:
    info_buf = io.StringIO()
    dataframe.info(buf=info_buf)
    _ = info_buf.getvalue()  # not used directly; we generate concise version below
    rows = []
    total = len(dataframe)
    for col in dataframe.columns:
        non_null = dataframe[col].notna().sum()
        null_pct = 100 * (1 - non_null / total)
        uniq = dataframe[col].nunique(dropna=True)
        rows.append(f"{col} | {dataframe[col].dtype} | {non_null} | {null_pct:.2f}% | {uniq}")
    header = "Column | DType | Non-Null | Null% | Unique\n------ | ----- | -------- | ----- | ------"
    concise = header + "\n" + "\n".join(rows)
    return concise

RANDOM_SEED = 42
SAMPLE_ROWS = 10
schema_concise = build_schema_summary(df)
sample_md = df.sample(n=SAMPLE_ROWS, random_state=RANDOM_SEED).to_markdown(index=False)

# 4. System Instruction (kept same semantic intent) ---------------------------
system_instruction = """
You are an expert data scientist specializing in data cleaning and preparation for machine learning. 
Your task is to perform exploratory data analysis (EDA), data cleaning and preprocessing for machine learning application. 
You will do this by generating structured insights, plans, and code.

Goals:
1. Perform a sharp initial EDA on a given dataset.
2. Propose an ordered data cleaning and preprocessing plan based on the EDA and best practices.
3. Following the data cleaning plan, produce fully executable, well-commented Python code.

Constraints:
- The created code must be executable without errors using the full dataset loaded from the provided CSV file path.
- Use Markdown headings exactly as requested.
"""

# 5. Generation Config -------------------------------------------------------
MODEL_NAME = "deepseek-reasoner"  
MAX_TOKENS = 32000  
TEMPERATURE = 0.15

# 6. Retry + Wrapper (Chat Completions) --------------------------------------

def call_model(messages: List[Dict[str, str]], max_tokens: int = MAX_TOKENS, temperature: float = TEMPERATURE):
    return client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        stream=False
    )

def generate_with_retry(prompt_text: str, label: str, max_retries: int = 3, backoff: float = 2.0):
    errors: List[str] = []
    for attempt in range(1, max_retries + 1):
        try:
            start = time.time()
            # Compose messages with system + user
            messages = [
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": prompt_text}
            ]
            response = call_model(messages)
            elapsed = time.time() - start
            text = response.choices[0].message.content.strip() if response.choices else ""
            if not text:
                raise ValueError("Empty response content.")
            print(f"[{label}] Success attempt={attempt} time={elapsed:.2f}s")
            return dict(text=text, raw=response, attempts=attempt, errors=errors)
        except Exception as e:
            err_msg = f"[{label}] Attempt {attempt} failed: {e}"
            print(err_msg)
            errors.append(err_msg)
            if attempt < max_retries:
                sleep_time = backoff ** (attempt - 1)
                time.sleep(sleep_time)
    print(f"[{label}] All attempts failed.")
    return dict(text="", raw=None, attempts=max_retries, errors=errors)

# 7. Prompts (EXACT TEXT PRESERVED from previous implementation) --------------
prompt_1_analysis = f"""
## Task 1: Initial Data Analysis

You are given:
### Concise Schema Summary
```
{schema_concise}
```

### Random Sample ({SAMPLE_ROWS} rows)
```
{sample_md}
```

### Directives
1. Use the schema and sample to perform a sharp exploratory data analysis (EDA).
2. Create a table of unique value counts for all columns in the dataset.
3. Create a table of high cardinality columns. Ensure to explain why the columns were selected.
4. Generate a data quality analysis containing accuracy, completeness, consistency, uniqueness, validity, and timeliness before data cleaning.

Output ONLY under heading:
## Analytical Insights
Use concise Markdown sections & tables. Avoid code.
"""



# 8. Execution Chain ---------------------------------------------------------
chain_artifacts: Dict[str, str] = {}

print("--- Chain Step 1: Initial Analysis ---")
analysis_result = generate_with_retry(prompt_1_analysis, label="analysis")

if not analysis_result["text"]:
    print("Aborting chain: analysis stage failed.")
else:
    chain_artifacts["analysis_md"] = analysis_result["text"]
    display(Markdown("### Analysis Received"))
    display(Markdown(chain_artifacts["analysis_md"]))

    # Step 2: Plan 
    prompt_2_plan = f"""
## Task 2: Data Cleaning Plan

Using the prior analysis:

### Analytical Insights
{chain_artifacts['analysis_md']}

Produce a prioritized, ordered bullet list of data cleaning and machine learning preprocessing steps.
Generate a data quality analysis containing accuracy, completeness, consistency, uniqueness, validity, and timeliness before data cleaning.
Do not perform train-test splits or modeling.
No code. Include: objective, columns impacted, and rationale.
Heading required: ## Data Cleaning Steps
"""
    print("\n--- Chain Step 2: Cleaning Plan ---")
    plan_result = generate_with_retry(prompt_2_plan, label="plan")

    if not plan_result["text"]:
        print("Aborting chain: plan stage failed.")
    else:
        chain_artifacts["plan_md"] = plan_result["text"]
        display(Markdown("### Cleaning Plan Received"))
        display(Markdown(chain_artifacts["plan_md"]))

        # Step 3: Code
        prompt_3_code = f"""
## Task 3: Data Cleaning Code

### Data Cleaning Steps
{chain_artifacts['plan_md']}

## Directives
1. Create fully executable Python code that implements the data cleaning plan developed in Task 2.
2. Your code should prepare one fully preprocessed dataframe. Do NOT perform any splitting or modeling.
3. Do not use sample data, only the full dataset loaded from the CSV file at path: {DATA_PATH}
4. Provide extremely detailed comments explaining each step of the code.


Return as ONE fenced Python code block under heading:
## Data Cleaning Code
"""
        print("\n--- Chain Step 3: Code Generation ---")
        code_result = generate_with_retry(prompt_3_code, label="code")

        if not code_result["text"]:
            print("Code generation failed.")
        else:
            chain_artifacts["code_md"] = code_result["text"]
            display(Markdown("### Data Cleaning Code Received"))
            display(Markdown(chain_artifacts["code_md"]))

        # Step 4: Validation
        prompt_4_validation = f"""
## Task 4: Data Cleaning Validation

### Data Cleaning Steps
{chain_artifacts['code_md']}

## Directives
1. Provide reasoning for any changes made from the original code.
2. Review the provided data cleaning code for correctness and completeness.
3. Ensure all steps from the cleaning plan are implemented in order.
4. Ensure the code is executable without errors using the full dataset loaded from the provided CSV file.
5. Provide extremely detailed comments explaining each step of the code.
6. Your code should prepare one fully preprocessed dataframe. Do NOT perform any splitting or modeling.
7. If the original code is correct, return it unchanged and note that no changes were needed.

Return as ONE fenced Python code block under heading:
## Data Cleaning Code - Validated
"""
        print("\n--- Chain Step 4: Code Validation ---")
        code_result = generate_with_retry(prompt_4_validation, label="validation")

        if not code_result["text"]:
            print("Validation check failed.")
        else:
            chain_artifacts["code_md"] = code_result["text"]
            display(Markdown("### Data Validation Received"))
            display(Markdown(chain_artifacts["code_md"]))


print("\n--- Prompt Chain Complete ---")

# 9. Export Artifacts --------------------------------------------------------
EXPORT_DIR = os.path.dirname(DATA_PATH)
export_path = "C:\\Users\\macdo\\Github\\VNFCDR-1\\Chris Code\\Generative AI Data Cleaning\\Machine Learning\\Step 3_DeepSeek Prompting\\UNSW_NB15_Train_Test_Concatenated\\DeepSeek_API_Exploratory_Analysis_Export.md"
try:
    with open(export_path, "w", encoding="utf-8") as f:
        if chain_artifacts:
            f.write("# EDA Prompt Chain Output (DeepSeek)\n\n")
            for k, v in chain_artifacts.items():
                pretty = k.replace("_md", "").capitalize()
                f.write(f"\n\n## {pretty}\n\n")
                f.write(v.strip() + "\n")
        else:
            f.write("No artifacts generated (chain failed).")
    print(f"Exported chain artifacts to: {export_path}")
except Exception as e:
    print(f"Export failed: {e}")

# 10. Diagnostic Recap -------------------------------------------------------

def print_diagnostics():
    print("\n=== Diagnostics Summary ===")
    print(f"Model used: {MODEL_NAME}")
    if analysis_result.get('errors'):
        print(f"Analysis errors: {len(analysis_result['errors'])}")
    if 'plan_result' in locals() and plan_result.get('errors'):
        print(f"Plan errors: {len(plan_result['errors'])}")
    if 'code_result' in locals() and code_result.get('errors'):
        print(f"Code errors: {len(code_result['errors'])}")
    print("Artifacts generated:", list(chain_artifacts.keys()))

print_diagnostics()

DeepSeek API client configured successfully.
Dataset loaded successfully. Shape: (257673, 42)
--- Chain Step 1: Initial Analysis ---
[analysis] Success attempt=1 time=55.74s


### Analysis Received

## Analytical Insights

### Dataset Overview
- **Total Rows**: 257,673
- **Total Columns**: 44
- **No Missing Data**: All columns show 0.00% null values
- **Mixed Data Types**: Primarily numerical (int64, float64) with one categorical column (attack_cat)
- **Target Variable**: `label` (binary classification) and `attack_cat` (multi-class classification)

### Unique Value Counts
| Column | Unique Values | Type |
|--------|---------------|------|
| id | 175,341 | Identifier |
| dur | 109,945 | Continuous |
| spkts | 646 | Discrete |
| dpkts | 627 | Discrete |
| sbytes | 9,382 | Discrete |
| dbytes | 8,653 | Discrete |
| rate | 115,763 | Continuous |
| sttl | 13 | Discrete |
| dttl | 9 | Discrete |
| sload | 121,356 | Continuous |
| dload | 116,380 | Continuous |
| sloss | 490 | Discrete |
| dloss | 476 | Discrete |
| sinpkt | 114,318 | Continuous |
| dinpkt | 110,270 | Continuous |
| sjit | 117,101 | Continuous |
| djit | 114,861 | Continuous |
| swin | 22 | Discrete |
| stcpb | 114,473 | Continuous |
| dtcpb | 114,187 | Continuous |
| dwin | 19 | Discrete |
| tcprtt | 63,878 | Continuous |
| synack | 57,366 | Continuous |
| ackdat | 53,248 | Continuous |
| smean | 1,377 | Discrete |
| dmean | 1,362 | Discrete |
| trans_depth | 14 | Discrete |
| response_body_len | 2,819 | Discrete |
| ct_srv_src | 57 | Discrete |
| ct_state_ttl | 7 | Discrete |
| ct_dst_ltm | 52 | Discrete |
| ct_src_dport_ltm | 52 | Discrete |
| ct_dst_sport_ltm | 35 | Discrete |
| ct_dst_src_ltm | 58 | Discrete |
| is_ftp_login | 4 | Binary-like |
| ct_ftp_cmd | 4 | Discrete |
| ct_flw_http_mthd | 11 | Discrete |
| ct_src_ltm | 52 | Discrete |
| ct_srv_dst | 57 | Discrete |
| is_sm_ips_ports | 2 | Binary |
| attack_cat | 10 | Categorical |
| label | 2 | Binary |

### High Cardinality Columns
| Column | Unique Count | Reason |
|--------|--------------|--------|
| id | 175,341 | Likely an identifier column with near-unique values |
| dur | 109,945 | Continuous duration measurements |
| rate | 115,763 | Continuous network rate values |
| sload | 121,356 | Continuous source load measurements |
| dload | 116,380 | Continuous destination load measurements |
| sinpkt | 114,318 | Continuous source inter-packet times |
| dinpkt | 110,270 | Continuous destination inter-packet times |
| sjit | 117,101 | Continuous source jitter measurements |
| djit | 114,861 | Continuous destination jitter measurements |
| stcpb | 114,473 | Continuous TCP base sequence numbers |
| dtcpb | 114,187 | Continuous TCP base sequence numbers |

**Selection Rationale**: These columns were selected because they exhibit very high unique value counts (≥100,000), indicating continuous numerical features that may require scaling/normalization for machine learning.

### Data Quality Analysis

**Accuracy**: 
- Numerical ranges appear plausible for network traffic data
- Extreme values observed (e.g., sload up to 800 million) may represent legitimate network anomalies or measurement errors

**Completeness**:
- Excellent completeness with 0% missing values across all columns
- No imputation required for missing data

**Consistency**:
- Data types align with expected measurement types
- Some zero values in network metrics (dbytes=0, dload=0) appear consistent with one-way traffic patterns
- TCP-related metrics show zero values when no TCP connection exists

**Uniqueness**:
- `id` column shows high uniqueness (68% unique), suggesting it may be an identifier
- Other columns show appropriate uniqueness levels for their respective measurement types

**Validity**:
- Binary columns (is_sm_ips_ports, label) contain only 0/1 values
- Categorical column (attack_cat) contains expected attack categories
- No obvious invalid values in numerical columns based on sample

**Timeliness**:
- Not applicable as this appears to be static network traffic data without temporal relevance


--- Chain Step 2: Cleaning Plan ---
[plan] Success attempt=1 time=42.33s


### Cleaning Plan Received

...

## Data Cleaning Code
```python
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy.stats import mstats  # For winsorization (capping extreme values)

# Load the dataset from the specified path
df = pd.read_csv(r"E:\Datasets\UNSW-NB15\Training and Testing Sets\UNSW_NB15_concatenated_dropped.csv")

# Create a copy of the dataframe to preserve original data
df_clean = df.copy()

### Step 1: Drop the 'id' column
# Rationale: The 'id' column is a near-unique identifier that doesn't contribute to predictive power
# and could cause model overfitting to specific observations
df_clean.drop(columns=['id'], inplace=True)

### Step 2: Harmonize target variables
# Rationale: Prevent data leakage by keeping only one target variable
# We choose to keep 'label' for binary classification and drop 'attack_cat'
df_clean.drop(columns=['attack_cat'], inplace=True)

### Step 3: Handle extreme values using winsorization
# Rationale: Cap extreme values to reduce skewness while preserving outlier information
# Define columns with extreme values based on EDA findings
extreme_value_cols = ['sload', 'dload', 'dur', 'rate', 'sinpkt', 'dinpkt', 
                     'sjit', 'djit', 'stcpb', 'dtcpb']

# Winsorize top and bottom 1% of values for each specified column
for col in extreme_value_cols:
    df_clean[col] = mstats.winsorize(df_clean[col], limits=[0.01, 0.01])

### Step 4: Scale high-cardinality continuous features
# Rationale: Standardize feature scales to prevent dominance by large-magnitude features
# Use RobustScaler due to presence of remaining outliers after winsorization
high_cardinality_features = [
    'dur', 'rate', 'sload', 'dload', 'sinpkt', 'dinpkt',
    'sjit', 'djit', 'stcpb', 'dtcpb'
]

# Initialize RobustScaler (less sensitive to outliers than StandardScaler)
scaler = RobustScaler()

# Apply scaling to specified features
df_clean[high_cardinality_features] = scaler.fit_transform(df_clean[high_cardinality_features])

### Step 5: Validate binary columns
# Rationale: Ensure binary columns contain expected values and are properly formatted
binary_columns = ['is_sm_ips_ports', 'is_ftp_login', 'ct_ftp_cmd']

for col in binary_columns:
    # Check unique values and data type
    unique_vals = df_clean[col].unique()
    print(f"Column '{col}' unique values: {unique_vals}")
    
    # Convert to integer if not already (ensures numerical representation)
    df_clean[col] = df_clean[col].astype(int)

### Step 6: Validate zero-value handling
# Rationale: Confirm zeros are legitimate values (not missing data) in network metrics
zero_value_columns = ['dbytes', 'dload']

for col in zero_value_columns:
    zero_count = (df_clean[col] == 0).sum()
    print(f"Column '{col}' has {zero_count} zero values ({(zero_count/len(df_clean))*100:.2f}%)")

# No imputation needed - zeros represent valid one-way network traffic

### Final Data Validation
print("\nFinal dataset shape:", df_clean.shape)
print("\nColumn dtypes:\n", df_clean.dtypes)
print("\nMissing values per column:\n", df_clean.isnull().sum())
print("\nFirst 5 rows of cleaned data:")
print(df_clean.head())

# The dataframe df_clean is now prepared for machine learning
# Features are scaled, extreme values are capped, and data types are validated
# Note: Target variable 'label' remains unchanged for binary classification
```


--- Chain Step 4: Code Validation ---
[validation] Success attempt=1 time=74.91s


### Data Validation Received

## Data Cleaning Code - Validated
```python
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler
from scipy.stats import mstats  # For winsorization (capping extreme values)

# Load the dataset from the specified path
df = pd.read_csv(r"E:\Datasets\UNSW-NB15\Training and Testing Sets\UNSW_NB15_concatenated_dropped.csv")

# Create a copy of the dataframe to preserve original data
df_clean = df.copy()

### Step 1: Drop the 'id' column
# Rationale: The 'id' column is a near-unique identifier that doesn't contribute to predictive power
# and could cause model overfitting to specific observations. This is a standard preprocessing step
# for ML datasets with record identifiers.
df_clean.drop(columns=['id'], inplace=True)

### Step 2: Harmonize target variables
# Rationale: Prevent data leakage by keeping only one target variable. 'attack_cat' contains
# multiclass attack categories while 'label' contains binary classification (normal vs attack).
# We choose 'label' for binary classification to maintain consistency with common network
# intrusion detection approaches.
df_clean.drop(columns=['attack_cat'], inplace=True)

### Step 3: Handle extreme values using winsorization
# Rationale: Cap extreme values to reduce skewness while preserving outlier information.
# Winsorization is preferred over removal for network traffic data where extreme values
# may represent legitimate but rare network events. Top and bottom 1% capping is a common
# approach that balances outlier mitigation with data preservation.
# Define columns with extreme values based on EDA findings
extreme_value_cols = ['sload', 'dload', 'dur', 'rate', 'sinpkt', 'dinpkt', 
                     'sjit', 'djit', 'stcpb', 'dtcpb']

# Winsorize top and bottom 1% of values for each specified column
# Note: mstats.winsorize returns a masked array, so we extract the underlying data
for col in extreme_value_cols:
    df_clean[col] = mstats.winsorize(df_clean[col], limits=[0.01, 0.01]).data

### Step 4: Scale high-cardinality continuous features
# Rationale: Standardize feature scales to prevent dominance by large-magnitude features.
# Use RobustScaler due to presence of remaining outliers after winsorization. RobustScaler
# uses median and IQR instead of mean and standard deviation, making it more resistant to
# outliers that may still exist after winsorization.
high_cardinality_features = [
    'dur', 'rate', 'sload', 'dload', 'sinpkt', 'dinpkt',
    'sjit', 'djit', 'stcpb', 'dtcpb'
]

# Initialize RobustScaler (less sensitive to outliers than StandardScaler)
scaler = RobustScaler()

# Apply scaling to specified features
df_clean[high_cardinality_features] = scaler.fit_transform(df_clean[high_cardinality_features])

### Step 5: Validate binary columns
# Rationale: Ensure binary columns contain expected values and are properly formatted.
# Network traffic binary flags should only contain 0/1 values. Converting to integer
# ensures proper handling by machine learning algorithms.
binary_columns = ['is_sm_ips_ports', 'is_ftp_login', 'ct_ftp_cmd']

for col in binary_columns:
    # Check unique values and data type
    unique_vals = df_clean[col].unique()
    print(f"Column '{col}' unique values: {unique_vals}")
    
    # Validate that only 0 and 1 values are present
    if set(unique_vals).issubset({0, 1}):
        # Convert to integer if not already (ensures numerical representation)
        df_clean[col] = df_clean[col].astype(int)
    else:
        print(f"Warning: Column '{col}' contains unexpected values: {unique_vals}")

### Step 6: Validate zero-value handling
# Rationale: Confirm zeros are legitimate values (not missing data) in network metrics.
# In network traffic data, zero values often represent one-way communication or missing
# responses, which are legitimate patterns in network behavior.
zero_value_columns = ['dbytes', 'dload']

for col in zero_value_columns:
    zero_count = (df_clean[col] == 0).sum()
    print(f"Column '{col}' has {zero_count} zero values ({(zero_count/len(df_clean))*100:.2f}%)")

# No imputation needed - zeros represent valid one-way network traffic patterns

### Final Data Validation
print("\nFinal dataset shape:", df_clean.shape)
print("\nColumn dtypes:\n", df_clean.dtypes)
print("\nMissing values per column:\n", df_clean.isnull().sum())
print("\nFirst 5 rows of cleaned data:")
print(df_clean.head())

# The dataframe df_clean is now prepared for machine learning
# Features are scaled, extreme values are capped, and data types are validated
# Note: Target variable 'label' remains unchanged for binary classification
```

**Changes Made:**
1. **Added .data extraction for winsorized values**: The `mstats.winsorize` function returns a masked array, which needed to be converted to a regular array using `.data` to ensure proper pandas DataFrame compatibility.

2. **Enhanced binary column validation**: Added a check to ensure all values in binary columns are strictly 0 or 1 before conversion, providing a warning if unexpected values are found.

3. **Improved comments and documentation**: Added more detailed explanations for each step, particularly explaining why specific techniques (winsorization, RobustScaler) were chosen for network traffic data.

**Validation Results:**
- The code follows a logical cleaning sequence
- All transformations are appropriate for network intrusion detection data
- The code handles edge cases and provides validation outputs
- The final dataframe maintains all necessary features for machine learning while addressing data quality issues
- No splitting or modeling is performed as requested


--- Prompt Chain Complete ---
Exported chain artifacts to: C:\Users\macdo\Github\VNFCDR-1\Chris Code\Generative AI Data Cleaning\Machine Learning\Step 3_DeepSeek Prompting\UNSW_NB15_Train_Test_Concatenated\DeepSeek_API_Exploratory_Analysis_Export.md

=== Diagnostics Summary ===
Model used: deepseek-reasoner
Artifacts generated: ['analysis_md', 'plan_md', 'code_md']


In [1]:
# Post Cleaning Data Quality Analysis

import os, io, time, traceback
import pandas as pd
from dotenv import load_dotenv
from IPython.display import display, Markdown
from typing import Dict, List
from openai import OpenAI

# 1. Environment / API Setup --------------------------------------------------
# Expect an env file containing DEEPSEEK_API_KEY
#ENV_PATH = r"/Users/sarahsetiawan/Desktop/VNFCDR-1/SarahCode/Generative_AI/GEMINI_API_KEY.env"  # <-- update if needed to point to DeepSeek key file
ENV_PATH = r"C:\\Users\\macdo\\Github\\VNFCDR-1\\Chris Code\\Generative AI Data Cleaning\\Machine Learning\\Step 3_DeepSeek Prompting\\DEEPSEEK_API_KEY.env"
load_dotenv(ENV_PATH)
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise RuntimeError("DEEPSEEK_API_KEY not found. Ensure the env file exists and key is set.")

client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com/v1")
print("DeepSeek API client configured successfully.")

# 2. Data Load ----------------------------------------------------------------
#DATA_PATH = r"/Users/sarahsetiawan/Desktop/VNFCDR-1/SarahCode/Sample_df/Code/MachineLearning/SampleData_API/CSVs/Representative_APISample_20000_2.csv"
DATA_PATH = r"C:\\Users\\macdo\\Github\\VNFCDR-1\\Chris Code\\Generative AI Data Cleaning\\Machine Learning\\Step 3_DeepSeek Prompting\\UNSW_NB15_Train_Test_Concatenated\\cleaned_data.csv"
try:
    df = pd.read_csv(DATA_PATH, low_memory=False)
except FileNotFoundError:
    raise FileNotFoundError(f"Dataset not found at {DATA_PATH}")
print(f"Dataset loaded successfully. Shape: {df.shape}")

# 3. Context Preparation (Token-Optimized) ------------------------------------


def build_schema_summary(dataframe: pd.DataFrame) -> str:
    info_buf = io.StringIO()
    dataframe.info(buf=info_buf)
    _ = info_buf.getvalue()  # not used directly; we generate concise version below
    rows = []
    total = len(dataframe)
    for col in dataframe.columns:
        non_null = dataframe[col].notna().sum()
        null_pct = 100 * (1 - non_null / total)
        uniq = dataframe[col].nunique(dropna=True)
        rows.append(f"{col} | {dataframe[col].dtype} | {non_null} | {null_pct:.2f}% | {uniq}")
    header = "Column | DType | Non-Null | Null% | Unique\n------ | ----- | -------- | ----- | ------"
    concise = header + "\n" + "\n".join(rows)
    return concise

RANDOM_SEED = 42
SAMPLE_ROWS = 10
schema_concise = build_schema_summary(df)
sample_md = df.sample(n=SAMPLE_ROWS, random_state=RANDOM_SEED).to_markdown(index=False)

# 4. System Instruction (kept same semantic intent) ---------------------------
system_instruction = """
You are an expert data scientist specializing in data cleaning and preparation for machine learning. 
Your task is to Generate a data quality analysis containing accuracy, completeness, consistency, uniqueness, validity, and timeliness on an imported file. 

Goals:
1. Perform a data quality analysis on a given dataset.

Constraints:
- Use Markdown headings exactly as requested.
"""

# 5. Generation Config -------------------------------------------------------
MODEL_NAME = "deepseek-reasoner"  
MAX_TOKENS = 32000  
TEMPERATURE = 0.15

# 6. Retry + Wrapper (Chat Completions) --------------------------------------

def call_model(messages: List[Dict[str, str]], max_tokens: int = MAX_TOKENS, temperature: float = TEMPERATURE):
    return client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
        stream=False
    )

def generate_with_retry(prompt_text: str, label: str, max_retries: int = 3, backoff: float = 2.0):
    errors: List[str] = []
    for attempt in range(1, max_retries + 1):
        try:
            start = time.time()
            # Compose messages with system + user
            messages = [
                {"role": "system", "content": system_instruction},
                {"role": "user", "content": prompt_text}
            ]
            response = call_model(messages)
            elapsed = time.time() - start
            text = response.choices[0].message.content.strip() if response.choices else ""
            if not text:
                raise ValueError("Empty response content.")
            print(f"[{label}] Success attempt={attempt} time={elapsed:.2f}s")
            return dict(text=text, raw=response, attempts=attempt, errors=errors)
        except Exception as e:
            err_msg = f"[{label}] Attempt {attempt} failed: {e}"
            print(err_msg)
            errors.append(err_msg)
            if attempt < max_retries:
                sleep_time = backoff ** (attempt - 1)
                time.sleep(sleep_time)
    print(f"[{label}] All attempts failed.")
    return dict(text="", raw=None, attempts=max_retries, errors=errors)

# 7. Prompts (EXACT TEXT PRESERVED from previous implementation) --------------
prompt_1_analysis = f"""
## Task 1: Data Quality Analysis

You are given:
### Concise Schema Summary
```
{schema_concise}
```

### Random Sample ({SAMPLE_ROWS} rows)
```
{sample_md}
```

### Directives
1. Generate a data quality analysis containing accuracy, completeness, consistency, uniqueness, validity, and timeliness on the impored file at {DATA_PATH}.

Output ONLY under heading:
## Analytical Insights
Use concise Markdown sections & tables. Avoid code.
"""
# 8. Execution Chain ---------------------------------------------------------
chain_artifacts: Dict[str, str] = {}

print("--- Chain Step 1: Data Quality Analysis ---")
analysis_result = generate_with_retry(prompt_1_analysis, label="analysis")

if not analysis_result["text"]:
    print("Aborting chain: analysis stage failed.")
else:
    chain_artifacts["analysis_md"] = analysis_result["text"]
    display(Markdown("### Analysis Received"))
    display(Markdown(chain_artifacts["analysis_md"]))

# 9. Export Artifacts --------------------------------------------------------
EXPORT_DIR = os.path.dirname(DATA_PATH)
export_path = "C:\\Users\\macdo\\Github\\VNFCDR-1\\Chris Code\\Generative AI Data Cleaning\\Machine Learning\\Step 3_DeepSeek Prompting\\UNSW_NB15_Train_Test_Concatenated\\DeepSeek_API_After_Cleaning_Analysis_Export.md"
try:
    with open(export_path, "w", encoding="utf-8") as f:
        if chain_artifacts:
            f.write("# EDA Prompt Chain Output (DeepSeek)\n\n")
            for k, v in chain_artifacts.items():
                pretty = k.replace("_md", "").capitalize()
                f.write(f"\n\n## {pretty}\n\n")
                f.write(v.strip() + "\n")
        else:
            f.write("No artifacts generated (chain failed).")
    print(f"Exported chain artifacts to: {export_path}")
except Exception as e:
    print(f"Export failed: {e}")

# 10. Diagnostic Recap -------------------------------------------------------

def print_diagnostics():
    print("\n=== Diagnostics Summary ===")
    print(f"Model used: {MODEL_NAME}")
    if analysis_result.get('errors'):
        print(f"Analysis errors: {len(analysis_result['errors'])}")
    if 'plan_result' in locals() and plan_result.get('errors'):
        print(f"Plan errors: {len(plan_result['errors'])}")
    if 'code_result' in locals() and code_result.get('errors'):
        print(f"Code errors: {len(code_result['errors'])}")
    print("Artifacts generated:", list(chain_artifacts.keys()))

print_diagnostics()

DeepSeek API client configured successfully.
Dataset loaded successfully. Shape: (930000, 41)
--- Chain Step 1: Data Quality Analysis ---
[analysis] Success attempt=1 time=50.21s


### Analysis Received

## Analytical Insights

### Completeness
- **Score: 100%**
- All 930,000 rows are fully populated across all 42 columns.
- No missing values detected in any field.

### Uniqueness
- High uniqueness variance observed:
  - Columns like `sjit` (575,603 unique), `sload` (575,603 unique), and `dur` (553,964 unique) exhibit near-cardinality, suggesting continuous measurements.
  - Low-cardinality columns include `is_ftp_login` (2 unique), `label` (2 unique), `ct_state_ttl` (7 unique), indicating categorical or binary flags.
- No duplicate rows inferred from sample dispersion.

### Validity
- **Critical issues detected:**
  - Negative values in `dur` (duration), `sload` (source load), `dload` (destination load), which are inherently non-negative metrics.
  - Negative values in `swin` (source TCP window) and `dwin` (destination TCP window), which should be ≥0.
  - `sttl` and `dttl` (time-to-live) include 0, which may be invalid for live packets.
  - `response_body_len` (response body length) contains negative values, which are illogical.
- These suggest potential data collection errors, normalization artifacts, or encoding issues.

### Consistency
- **Internal logic violations:**
  - Rows with `dpkts=0` but `dbytes>0` (e.g., sample row 1: `dpkts=0`, `dbytes=0` is consistent, but other rows may not be).
  - `spkts` and `sloss` should satisfy `sloss ≤ spkts`, but sample shows `spkts=2` and `sloss=0` (valid), though other combinations may breach this.
  - `rate` (flow rate) negative in some rows, which may conflict with positive `sbytes`/`dbytes`.
- Protocol-specific fields (e.g., `tcprtt`, `synack`) show negative values, inconsistent with TCP timing semantics.

### Accuracy
- Cannot be fully assessed without ground truth, but anomalies suggest inaccuracies:
  - Negative network metrics (e.g., load, jitter, packet counts) are not physically plausible.
  - Extreme values in normalized fields (e.g., `stcpb` ≈ -0.735, `dtcpb` ≈ -0.735) may indicate erroneous transformations or outliers.
- Labels (`attack_cat`, `label`) appear valid with expected ranges (0-10 and 0-1).

### Timeliness
- No temporal columns (e.g., timestamps) present in the schema.
- Dataset appears static for modeling purposes, so timeliness is not directly applicable.

### Summary Table
| Dimension      | Score | Issues                                                                 |
|----------------|-------|------------------------------------------------------------------------|
| Completeness   | 100%  | None                                                                   |
| Uniqueness     | -     | Expected variance for continuous vs. categorical fields                |
| Validity       | Low   | Negative values in non-negative metrics (dur, load, TTL, win, bytes)   |
| Consistency    | Low   | Logic conflicts (e.g., dpkts=0 vs. dbytes>0), negative rates/timings   |
| Accuracy       | Medium| Suspect due to invalid values; requires domain verification            |
| Timeliness     | N/A   | No time-based columns                                                  |

Exported chain artifacts to: C:\Users\macdo\Github\VNFCDR-1\Chris Code\Generative AI Data Cleaning\Machine Learning\Step 3_DeepSeek Prompting\UNSW_NB15_Train_Test_Concatenated\DeepSeek_API_After_Cleaning_Analysis_Export.md

=== Diagnostics Summary ===
Model used: deepseek-reasoner
Artifacts generated: ['analysis_md']
